<h1><center>🏀 Mania 2022 - Men's 🏀</center></h1>

<h2>Table of Content</h2>

* [Introduction 📒](#0)
    - [Librairies import and load the dataset 📚](#0.1) 
* [Understand the Data](#1)
* [Exploratory Data Analysis (EDA) 📊](#2)
    - [Checking missing Data & Split categorical / Numerical features](#2.1)
* [Data Preprocessing](#3)
    - [Deal with redundant and useless features](#3.1)
    - [Dealing with Outliers](#3.2)
    - [Treating Missing Values](#3.3)
    - [Feature Engineering](#3.3)
* [Modeling and training](#4)
* [Conclusion](#5)




<a id="0"></a> <br>

# 0 Introduction 📒

In my data science learning journey I take part in this competition to try my best and get a good score. By competing I will learn and put into practice several tools that I have learned here in Kaggle.

Thus, in this notebook I will perform the following tasks :

1. Understand the data of this competition
2. Explore the data with data visualisation tools
3. Feature engineering
4. Modeling & training

Since the real competition is forecasting the 2022 results, I will use the time before kaggle accepts the prediction for 2022 (**Monday, Mar 14**) to understand the data, read how kagglers handle this competition in 2021 and try my best to adapt things to this competition.


<a id="0.1"></a> <br>

## 0.1 Import Libraries and Load the Dataset 📚

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import os
import seaborn as sns
from numpy import mean,std
from sklearn.metrics import confusion_matrix
from sklearn.metrics import plot_confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.preprocessing import MinMaxScaler

from sklearn.utils import class_weight
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier

import xgboost as xgb
from sklearn.metrics import ConfusionMatrixDisplay


from sklearn.model_selection import GroupKFold, KFold
from sklearn.metrics import log_loss
import lightgbm as lgb

import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns; sns.set()

from sklearn.model_selection import GroupKFold, KFold
from sklearn.metrics import log_loss
import lightgbm as lgb


import warnings
warnings.filterwarnings('ignore')

In [ ]:
#DATA_DIR = '../input/mens-march-mania-2022/MDataFiles_Stage1/'
DATA_DIR = '../input/mens-march-mania-2022/MDataFiles_Stage2/'

STAGE_1 = False # This needs to be False when it's stage 2 

In [ ]:
teams = pd.read_csv(DATA_DIR +"MTeams.csv")
seasons = pd.read_csv(DATA_DIR +"MSeasons.csv")

MRSCResults = pd.read_csv(DATA_DIR +'MRegularSeasonCompactResults.csv')
cities = pd.read_csv(DATA_DIR +"Cities.csv")
conferences = pd.read_csv(DATA_DIR +"Conferences.csv")
gameCities = pd.read_csv(DATA_DIR +"MGameCities.csv")
MMOrdinals = pd.read_csv('../input/mens-march-mania-2022/MDataFiles_Stage1/MMasseyOrdinals.csv')
MTCResults = pd.read_csv(DATA_DIR +"MNCAATourneyCompactResults.csv")
MRSDResults = pd.read_csv(DATA_DIR +"MRegularSeasonDetailedResults.csv")
MTDResults = pd.read_csv(DATA_DIR +"MNCAATourneyDetailedResults.csv")
tourneySeeds = pd.read_csv(DATA_DIR +"MNCAATourneySeeds.csv")



<a id="1"></a> <br>

# 1 Understand the data

In this section I will explore a bit the data and try to understand the different features in this dataset. The idea is to treat numerical and categorical independently since the tools to analyse each type of data are not the same.

> The data section on the competition webpage nicely describes the different files and features : https://www.kaggle.com/c/mens-march-mania-2022/data

**Submission file and match since 2016**

In [ ]:
sub = pd.read_csv('../input/mens-march-mania-2022/MDataFiles_Stage1/MSampleSubmissionStage1.csv')
sub[['year', 'Team_1', 'Team_2']] = pd.DataFrame(sub['ID'].str.split('_').values.tolist(), index=sub.index)
sub['year'] = pd.to_numeric(sub.year)
sub.head()

In [ ]:
sub['year'].value_counts()

#### **Teams**

In [ ]:
teams.head()

In [ ]:
plt.figure(figsize=(15,8))
sns.countplot(teams["FirstD1Season"])

Most teams have been in the D1 before 85 and only a few are "recent" teams.

#### **Seasons**

In [ ]:
seasons.head()

In [ ]:
reg = ["RegionW","RegionX","RegionY","RegionZ"]
fig = plt.figure(figsize=(13,6))
for index,i in enumerate(reg):
    plt.subplot(2,2,index+1)
    sns.countplot(seasons[i])
    
fig.tight_layout(pad=1.0)



#### **Compacted Results**

In [ ]:
MRSCResults.head()

In [ ]:
MRSCResults['PointDiff'] = MRSCResults['WScore'] - MRSCResults['LScore']
plt.figure(figsize=(15,8))
sns.countplot(MRSCResults[MRSCResults["PointDiff"] < 50]["PointDiff"])

In [ ]:
MRSCResults.describe()

I will create a summary of a season with some statistics (Mean,Median) and some other stats

In [ ]:
summaries = MRSCResults[['Season', 
    'WScore', 
    'LScore', 
    'NumOT', 
    'PointDiff']].groupby('Season').agg(['min', 'max', 'mean', 'median'])

summaries.columns = ['_'.join(col).strip() for col in summaries.columns.values]
summaries.head()

In [ ]:
cities.head()

In [ ]:
conferences.head()

In [ ]:
gameCities.head()

In [ ]:
MMOrdinals.head()

#### **Detailed results**

In [ ]:
MRSDResults.head()

In [ ]:
MTDResults.head()

In [ ]:
tourneySeeds.head()

<a id="2"></a> <br>
# 2 Exploratory Data Analysis (EDA) 📊

In this section I will go more in details through the different files.


In [ ]:
MRSCResults.head()

In [ ]:
teams.head()

In [ ]:
MRSDResults[['WTeamID','LTeamID','WScore','LScore','WFGM','LFGM']]

In [ ]:
MRSCResults['counter'] = 1
win_df = MRSCResults.groupby('WTeamID')['counter'].count()
win_df.rename("seasonWin",inplace=True)
lose_df = MRSCResults.groupby('LTeamID')['counter'].count()
lose_df.rename("seasonLose",inplace=True)

teams = pd.merge(teams, win_df, left_on="TeamID", right_on = "WTeamID")
teams = pd.merge(teams, lose_df, left_on="TeamID", right_on = "LTeamID")



In [ ]:
MTCResults['counter'] = 1
win_df = MTCResults.groupby('WTeamID')['counter'].count()
win_df.rename("tourneyWin",inplace=True)
lose_df = MTCResults.groupby('LTeamID')['counter'].count()
lose_df.rename("tourneyLose",inplace=True)

teams = pd.merge(teams, win_df, left_on="TeamID", right_on = "WTeamID")
teams = pd.merge(teams, lose_df, left_on="TeamID", right_on = "LTeamID")

In [ ]:
teams

In [ ]:
teams.sort_values("seasonWin",ascending = False).head(10)

In [ ]:
teams.sort_values("seasonLose",ascending = False).head(10)

I will compute the median and mode of these stats for each teams in regular Season as well as in tournament and for winning and losing matches
* WFGM - field goals made (by the winning team)
* WFGA - field goals attempted (by the winning team)
* WFGM3 - three pointers made (by the winning team)
* WFGA3 - three pointers attempted (by the winning team)
* WFTM - free throws made (by the winning team)
* WFTA - free throws attempted (by the winning team)
* WOR - offensive rebounds (pulled by the winning team)
* WDR - defensive rebounds (pulled by the winning team)
* WAst - assists (by the winning team)
* WTO - turnovers committed (by the winning team)
* WStl - steals (accomplished by the winning team)
* WBlk - blocks (accomplished by the winning team)
* WPF - personal fouls committed (by the winning team)

In [ ]:
win_or_lose = ['W','L']
features = ['FGM','FGA','FGM3','FGA3','FTM','FTA','OR','DR','Ast','TO','Stl','Blk','PF']


# tmp = MRSDResults.groupby('{}TeamID'.format(w_l))['{}{}'.format(w_l,feature)].mean().reset_index(name ='{}{}_mean'.format(w_l,feature))

for w_l in win_or_lose:
    for feature in features:
        tmp = MRSDResults.groupby('{}TeamID'.format(w_l)).agg(**{'Season_{}{}_mean'.format(w_l,feature) : ('{}{}'.format(w_l,feature),'mean'),
                                                               'Season_{}{}_median'.format(w_l,feature) : ('{}{}'.format(w_l,feature),'median')})
        
        
        teams = pd.merge(teams, tmp, left_on="TeamID", right_on = "{}TeamID".format(w_l))

teams.head()

In [ ]:
teams.sort_values("seasonWin",ascending = False).head(10)

In [ ]:
teams[['TeamName','seasonWin','seasonLose','Season_WFGM_mean','Season_LFGM_mean']].sort_values("seasonWin",ascending = False).head(10)

In [ ]:
win_or_lose = ['W','L']
features = ['FGM','FGA','FGM3','FGA3','FTM','FTA','OR','DR','Ast','TO','Stl','Blk','PF']

fig = plt.figure(figsize=(20,60))

a = 0
for i in ('mean','median'):
    for feature in features:
            plt.subplot(13,4,a+1)
            ax = sns.regplot(data = teams, x = 'Season_W{}_{}'.format(feature,i), y = 'seasonWin',label = "Win")
            ax  = sns.regplot(data = teams, x = 'Season_L{}_{}'.format(feature,i), y = 'seasonWin', label = 'Lose',ax=ax)
            ax.legend(loc = 2)
            ax.set_xlabel("{}_{}".format(feature,i))
            a+=1

In [ ]:
win_or_lose = ['W','L']
features = ['FGM','FGA','FGM3','FGA3','FTM','FTA','OR','DR','Ast','TO','Stl','Blk','PF']

fig = plt.figure(figsize=(20,60))

a = 0
for i in ('mean','median'):
    for feature in features:
            plt.subplot(13,4,a+1)
            ax = sns.regplot(data = teams, x = 'Season_W{}_{}'.format(feature,i), y = 'seasonWin',label = "Win")
            ax  = sns.regplot(data = teams, x = 'Season_L{}_{}'.format(feature,i), y = 'seasonLose', label = 'Lose',ax=ax)
            ax.legend(loc = 2)
            ax.set_xlabel("{}_{}".format(feature,i))
            ax.set_xlabel("season Win/Lose")
            a+=1

There are clear correlations between some stats and the number of wins (or lose) a team have during a season. The cluster between losing and winning stats for some features are really disctinct so it will be an important feature to chose for training.

#### **Tourney detailled results**

In [ ]:
win_or_lose = ['W','L']
features = ['FGM','FGA','FGM3','FGA3','FTM','FTA','OR','DR','Ast','TO','Stl','Blk','PF']

for w_l in win_or_lose:
    for feature in features:
        tmp = MTDResults.groupby('{}TeamID'.format(w_l)).agg(**{'Tourney_{}{}_mean'.format(w_l,feature) : ('{}{}'.format(w_l,feature),'mean'),
                                                               'Tourney_{}{}_median'.format(w_l,feature) : ('{}{}'.format(w_l,feature),'median')})
        
        
        teams = pd.merge(teams, tmp, left_on="TeamID", right_on = "{}TeamID".format(w_l))

teams.head()

In [ ]:
win_or_lose = ['W','L']
features = ['FGM','FGA','FGM3','FGA3','FTM','FTA','OR','DR','Ast','TO','Stl','Blk','PF']

fig = plt.figure(figsize=(20,60))

a = 0
for i in ('mean','median'):
    for feature in features:
            plt.subplot(13,4,a+1)
            ax = sns.regplot(data = teams, x = 'Tourney_W{}_{}'.format(feature,i), y = 'tourneyWin',label = "Win")
            ax  = sns.regplot(data = teams, x = 'Tourney_L{}_{}'.format(feature,i), y = 'tourneyLose', label = 'Lose',ax=ax)
            ax.legend(loc = 2)
            ax.set_xlabel("{}_{}".format(feature,i))
            ax.set_xlabel("tourney Win/Lose")
            a+=1

#### **Calculating Win percentage**

I'll compute the win percentage per teams when playing at home (H), away (A) or on a neutral court 

In [ ]:
A_w = MRSCResults[MRSCResults.WLoc == 'A']\
    .groupby(['Season','WTeamID'])['WTeamID'].count().to_frame()\
    .rename(columns={"WTeamID": "win_A"})
N_w = MRSCResults[MRSCResults.WLoc == 'N']\
    .groupby(['Season','WTeamID'])['WTeamID'].count().to_frame()\
    .rename(columns={"WTeamID": "win_N"})
H_w = MRSCResults[MRSCResults.WLoc == 'H']\
    .groupby(['Season','WTeamID'])['WTeamID'].count().to_frame()\
    .rename(columns={"WTeamID": "win_H"})
win = A_w.join(N_w, how='outer').join(H_w, how='outer').fillna(0)

H_l = MRSCResults[MRSCResults.WLoc == 'A']\
    .groupby(['Season','LTeamID'])['LTeamID'].count().to_frame()\
    .rename(columns={"LTeamID": "lost_H"})
N_l = MRSCResults[MRSCResults.WLoc == 'N']\
    .groupby(['Season','LTeamID'])['LTeamID'].count().to_frame()\
    .rename(columns={"LTeamID": "lost_N"})
A_l = MRSCResults[MRSCResults.WLoc == 'H']\
    .groupby(['Season','LTeamID'])['LTeamID'].count().to_frame()\
    .rename(columns={"LTeamID": "lost_A"})
lost = A_l.join(N_l, how='outer').join(H_l, how='outer').fillna(0)

win.index = win.index.rename(['Season', 'TeamID'])
lost.index = lost.index.rename(['Season', 'TeamID'])
wl = win.join(lost, how='outer').reset_index()
wl['win_pct_A'] = wl['win_A'] / (wl['win_A'] + wl['lost_A'])
wl['win_pct_N'] = wl['win_N'] / (wl['win_N'] + wl['lost_N'])
wl['win_pct_H'] = wl['win_H'] / (wl['win_H'] + wl['lost_H'])
wl['win_pct_All'] = (wl['win_A'] + wl['win_N'] + wl['win_H']) / \
    (wl['win_A'] + wl['win_N'] + wl['win_H'] + wl['lost_A']\
     + wl['lost_N'] + wl['lost_H'])

del A_w, N_w, H_w, H_l, N_l, A_l, win, lost

In [ ]:
wl.head()

In [ ]:
MRSCResults['relScore'] = MRSCResults.WScore - MRSCResults.LScore

w_scr = MRSCResults.loc[:, ['Season', 'WTeamID', 'WScore', 'WLoc','relScore']]
w_scr.columns = ['Season', 'TeamID','Score','Loc','relScore']
l_scr = MRSCResults.loc[:, ['Season', 'LTeamID', 'LScore', 'WLoc','relScore']]
l_scr['WLoc'] = l_scr.WLoc.apply(lambda x: 'H' if x == 'A' else 'A' \
                                 if x == 'H' else 'N')
l_scr['relScore'] = -1 * l_scr.relScore 
l_scr.columns = ['Season', 'TeamID','Score','Loc','relScore']
wl_scr = pd.concat([w_scr,l_scr])

A_scr = wl_scr[wl_scr.Loc == 'A'].groupby(['Season','TeamID'])\
        ['Score','relScore'].mean()\
        .rename(columns={"Score": "Score_A", "relScore": "relScore_A"})
N_scr = wl_scr[wl_scr.Loc == 'N'].groupby(['Season','TeamID'])\
        ['Score','relScore'].mean()\
        .rename(columns={"Score": "Score_N", "relScore": "relScore_N"})
H_scr = wl_scr[wl_scr.Loc == 'H'].groupby(['Season','TeamID'])\
        ['Score','relScore'].mean()\
        .rename(columns={"Score": "Score_H", "relScore": "relScore_H"})
All_scr = wl_scr.groupby(['Season','TeamID'])['Score','relScore']\
    .mean().rename(columns={"Score": "Score_All", "relScore": "relScore_All"})
scr = A_scr.join(N_scr, how='outer').join(H_scr, how='outer')\
    .join(All_scr, how='outer').fillna(0).reset_index()

del w_scr, l_scr, wl_scr, A_scr, H_scr, N_scr, All_scr

In [ ]:
scr.describe()

In [ ]:
w = MRSDResults.loc[:, ['Season', 'WTeamID', 'WFGM','WFGA','WFGM3'
                               ,'WFGA3','WFTM','WFTA','WOR','WDR','WAst',
                               'WTO','WStl','WBlk','WPF']]
w.columns = ['Season', 'TeamID', 'FGM','FGA','FGM3','FGA3','FTM','FTA','OR','DR',
             'Ast','TO','Stl','Blk','PF']
l = MRSDResults.loc[:, ['Season', 'LTeamID', 'LFGM','LFGA','LFGM3',
                               'LFGA3','LFTM','LFTA','LOR','LDR','LAst',
                               'LTO','LStl','LBlk','LPF']]
l.columns = ['Season', 'TeamID', 'FGM','FGA','FGM3','FGA3','FTM','FTA','OR','DR',
             'Ast','TO','Stl','Blk','PF']

detail = pd.concat([w,l])
detail['goal_rate'] = detail.FGM / detail.FGA 
detail['3p_goal_rate'] = detail.FGM3 / detail.FGA3  
detail['ft_goal_rate'] = detail.FTM  / detail.FTA  

dt = detail.groupby(['Season','TeamID'])['FGM','FGA','FGM3','FGA3','FTM','FTA',
                                         'OR','DR','Ast','TO','Stl','Blk','PF',
                                          'goal_rate', '3p_goal_rate',
                                         'ft_goal_rate']\
                                        .mean().fillna(0).reset_index()

del w, l, detail

In [ ]:
dt.head()

In [ ]:
MOR_127_128 = MMOrdinals[(MMOrdinals.SystemName == 'MOR') & \
                ((MMOrdinals.RankingDayNum == 127) \
                 | (MMOrdinals.RankingDayNum == 128))]\
                [['Season','TeamID','OrdinalRank']]
MOR_50_51 = MMOrdinals[(MMOrdinals.SystemName == 'MOR') & \
                ((MMOrdinals.RankingDayNum == 50) \
                 | (MMOrdinals.RankingDayNum == 51))]\
                [['Season','TeamID','OrdinalRank']]
MOR_15_16 = MMOrdinals[(MMOrdinals.SystemName == 'MOR') & \
                ((MMOrdinals.RankingDayNum == 15) \
                 | (MMOrdinals.RankingDayNum == 16))]\
                [['Season','TeamID','OrdinalRank']]

MOR_127_128 = MOR_127_128.rename(columns={'OrdinalRank':'OrdinalRank_127_128'})
MOR_50_51 = MOR_50_51.rename(columns={'OrdinalRank':'OrdinalRank_50_51'})
MOR_15_16 = MOR_15_16.rename(columns={'OrdinalRank':'OrdinalRank_15_16'})

MOR = MOR_127_128.merge(MOR_50_51, how='left', on=['Season','TeamID'])\
        .merge(MOR_15_16, how='left', on=['Season','TeamID'])

## normalizing Rank values by its season maxium as it varies by seasons
MOR_max = MOR.groupby('Season')['OrdinalRank_127_128','OrdinalRank_50_51',
                                'OrdinalRank_15_16'].max().reset_index()
MOR_max.columns = ['Season', 'maxRank_127_128', 'maxRank_50_51', 'maxRank_15_16']

MOR_tmp = MMOrdinals[(MMOrdinals.SystemName == 'MOR') \
                     & (MMOrdinals.RankingDayNum < 133)]
MOR_stats = MOR_tmp.groupby(['Season','TeamID'])['OrdinalRank']\
            .agg(['max','min','std','mean']).reset_index()
MOR_stats.columns = ['Season','TeamID','RankMax','RankMin','RankStd','RankMean']

MOR = MOR.merge(MOR_max, how='left', on='Season')\
        .merge(MOR_stats, how='left', on=['Season','TeamID'])
MOR['OrdinalRank_127_128'] = MOR['OrdinalRank_127_128'] / MOR['maxRank_127_128']
MOR['OrdinalRank_50_51'] = MOR['OrdinalRank_50_51'] / MOR['maxRank_50_51']
MOR['OrdinalRank_15_16'] = MOR['OrdinalRank_15_16'] / MOR['maxRank_15_16']
MOR['RankTrans_50_51_to_127_128'] = MOR['OrdinalRank_127_128'] \
                                    - MOR['OrdinalRank_50_51']
MOR['RankTrans_15_16_to_127_128'] = MOR['OrdinalRank_127_128'] \
                                    - MOR['OrdinalRank_15_16']


MOR.drop(['OrdinalRank_50_51','OrdinalRank_15_16', 'maxRank_50_51'
          ,'maxRank_15_16'],axis=1, inplace=True)

del MOR_127_128, MOR_50_51, MOR_15_16, MOR_max, MOR_tmp, MOR_stats

In [ ]:
wl_1 = wl.loc[:,['Season','TeamID','win_pct_A','win_pct_N',
                 'win_pct_H','win_pct_All']]
wl_1.columns = [str(col) + '_1' if col not in ['Season','TeamID'] \
                else str(col) for col in wl_1.columns ]

wl_2 = wl.loc[:,['Season','TeamID','win_pct_A','win_pct_N',
                 'win_pct_H','win_pct_All']]
wl_2.columns = [str(col) + '_2' if col not in ['Season','TeamID'] \
                else str(col) for col in wl_2.columns ]

scr_1 = scr.copy()
scr_1.columns = [str(col) + '_1' if col not in ['Season','TeamID'] \
                 else str(col) for col in scr_1.columns ]

scr_2 = scr.copy()
scr_2.columns = [str(col) + '_2' if col not in ['Season','TeamID'] \
                 else str(col) for col in scr_2.columns ]

dt_1 = dt.copy()
dt_1.columns = [str(col) + '_1' if col not in ['Season','TeamID'] \
                else str(col) for col in dt_1.columns ]

dt_2 = dt.copy()
dt_2.columns = [str(col) + '_2' if col not in ['Season','TeamID'] \
                else str(col) for col in dt_2.columns ]

MOR_1 = MOR.copy()
MOR_1.columns = [str(col) + '_1' if col not in ['Season','TeamID'] \
                 else str(col) for col in MOR_1.columns ]

MOR_2 = MOR.copy()
MOR_2.columns = [str(col) + '_2' if col not in ['Season','TeamID'] \
                 else str(col) for col in MOR_2.columns ]

In [ ]:
tourney1 = MTCResults.loc[:, ['Season','WTeamID','LTeamID']]
tourney1.columns = ['Season','TeamID1','TeamID2']
tourney1['result'] = 1

tourney2 = MTCResults.loc[:, ['Season','LTeamID','WTeamID']]
tourney2.columns = ['Season','TeamID1','TeamID2']
tourney2['result'] = 0

tourney = pd.concat([tourney1, tourney2])
del tourney1, tourney2

In [ ]:
def merge_data(df):

    df = df.merge(wl_1, how='left', left_on=['Season','TeamID1'],
                  right_on=['Season','TeamID'])
    df = df.merge(wl_2, how='left', left_on=['Season','TeamID2'],
                  right_on=['Season','TeamID'])
    df = df.drop(['TeamID_x','TeamID_y'], axis=1)


    df = df.merge(scr_1, how='left', left_on=['Season','TeamID1'],
                  right_on=['Season','TeamID'])
    df = df.merge(scr_2, how='left', left_on=['Season','TeamID2'],
                  right_on=['Season','TeamID'])
    df = df.drop(['TeamID_x','TeamID_y'], axis=1)

    df = df.merge(dt_1, how='left', left_on=['Season','TeamID1'],
                  right_on=['Season','TeamID'])
    df = df.merge(dt_2, how='left', left_on=['Season','TeamID2'],
                  right_on=['Season','TeamID'])
    
    df = df.drop(['TeamID_x','TeamID_y'], axis=1)

    df = df.merge(MOR_1, how='left', left_on=['Season','TeamID1'],
                  right_on=['Season','TeamID'])
    df = df.merge(MOR_2, how='left', left_on=['Season','TeamID2'],
                  right_on=['Season','TeamID'])
    df = df.drop(['TeamID_x','TeamID_y'], axis=1)

    df['OrdinalRank_127_128_diff'] = df['OrdinalRank_127_128_1'] \
        - df['OrdinalRank_127_128_2']
    
    df['magic1'] = df['OrdinalRank_127_128_diff'] - df['RankMean_1']
    df['magic2'] = df['RankMean_1'] - df['RankMean_2']
    df['magic3'] = df['OrdinalRank_127_128_diff'] - df['RankMean_2']
    
    df['magic11'] = df['OrdinalRank_127_128_diff'] * df['RankMean_1']
    df['magic21'] = df['RankMean_1'] * df['RankMean_2']
    df['magic31'] = df['OrdinalRank_127_128_diff'] * df['RankMean_2']
    
    df['magic12'] = df['OrdinalRank_127_128_diff'] / df['RankMean_1']
    df['magic22'] = df['RankMean_1'] / df['RankMean_2']
    df['magic32'] = df['OrdinalRank_127_128_diff'] / df['RankMean_2']

    df = df.fillna(-1)
    
    for col in df.columns:
        if (df[col] == np.inf).any() or (df[col] == -np.inf).any():
            df[col][(df[col] == np.inf) | (df[col] == -np.inf)] = -1
    
    return df

tourney = merge_data(tourney)
tourney = tourney.loc[tourney.Season >= 2003,:].reset_index(drop=True)

if STAGE_1:
    tourney = tourney.loc[tourney.Season < 2016, :]

In [ ]:
if STAGE_1:
    MSampleSubmission = pd.read_csv(DATA_DIR + 'MSampleSubmissionStage1.csv')
else:
    MSampleSubmission = pd.read_csv(DATA_DIR + 'MSampleSubmissionStage2.csv') # put stage 2 submission file link here

test1 = MSampleSubmission.copy()
test1['Season'] = test1.ID.apply(lambda x: int(x[0:4]))
test1['TeamID1'] = test1.ID.apply(lambda x: int(x[5:9]))
test1['TeamID2'] = test1.ID.apply(lambda x: int(x[10:14]))

test2 = MSampleSubmission.copy()
test2['Season'] = test2.ID.apply(lambda x: int(x[0:4]))
test2['TeamID1'] = test2.ID.apply(lambda x: int(x[10:14]))
test2['TeamID2'] = test2.ID.apply(lambda x: int(x[5:9]))

test = pd.concat([test1,test2]).drop(['Pred'], axis=1)
test = merge_data(test)

In [ ]:
X = tourney.drop(['Season','TeamID1','TeamID2','result'], axis=1)
y = tourney["result"]
s = tourney["Season"]

X_test = test.drop(['ID', 'Season','TeamID1','TeamID2'], axis=1)

<a id="4"></a> <br>
# 4 Modeling and training

In [ ]:
def model_training(X, y, cv, groups, params, metric, early_stopping=10, \
    plt_iter=True, X_test=[], cat_features=[]):

    feature_importance = pd.DataFrame()
    val_scores=[]
    train_evals=[]
    valid_evals=[]

    if len(X_test) > 0:
        test_pred=np.zeros(len(X_test))

    for idx, (train_index, val_index) in enumerate(cv.split(X, y, groups)):

        print("###### fold %d ######" % (idx+1))
        X_train, X_val = X.iloc[train_index], X.iloc[val_index]
        y_train, y_val = y.iloc[train_index], y.iloc[val_index]

        model = lgb.LGBMClassifier(**params)

        model.fit(X_train, y_train,
                  eval_set=[(X_train, y_train), (X_val, y_val)],
                  early_stopping_rounds=early_stopping,
                  verbose=20
                  #categorical_feature=list(cate_ft_lst),
                  )

        val_scores.append(model.best_score_['valid_1'][metric])
        train_evals.append(model.evals_result_['training'][metric])
        valid_evals.append(model.evals_result_['valid_1'][metric])

        if len(X_test) > 0:
            test_pred = test_pred + model.predict_proba(X_test, num_iteration=model.best_iteration_)[:,1]

        fold_importance = pd.DataFrame()
        fold_importance["feature"] = X_train.columns
        fold_importance["importance"] = model.feature_importances_
        fold_importance["fold"] = idx+1
        feature_importance = pd.concat([feature_importance, fold_importance]
                                       , axis=0)

    if plt_iter:
        
        fig, axs = plt.subplots(2, 2, figsize=(9,6))
        
        for i, ax in enumerate(axs.flatten()):
            ax.plot(train_evals[i], label='training')
            ax.plot(valid_evals[i], label='validation')
            ax.set(xlabel='interations', ylabel=f'{metric}')
            ax.set_title(f'fold {i+1}', fontsize=12)
            ax.legend(loc='upper right', prop={'size': 9})
        fig.tight_layout()
        plt.show()
    
    print('### CV scores by fold ###')
    for i in range(cv.get_n_splits(X)):
        print(f'fold {i+1}: {val_scores[i]:.4f}')
    print('CV mean score: {0:.4f}, std: {1:.4f}.'\
          .format(np.mean(val_scores), np.std(val_scores)))
    
    feature_importance = feature_importance[["feature", "importance"]]\
                         .groupby("feature").mean().sort_values(
                         by="importance", ascending=False)
    feature_importance.reset_index(inplace=True)

    if len(X_test) > 0:
        test_pred = test_pred / cv.get_n_splits(X)
        return feature_importance, test_pred
    else:
        return feature_importance

In [ ]:
lgb_params = {'objective': 'binary',
              'metric': 'binary_logloss',
              'boosting': 'gbdt',
              'num_leaves': 31,
              'feature_fraction': 0.8,
              'bagging_fraction': 0.8,
              'bagging_freq': 5,
              'learning_rate': 0.1,
              'n_estimators': 1000,
}

In [ ]:
%%time

N_FOLDS = 10

group_kfold = GroupKFold(n_splits=N_FOLDS)

feature_importance, test_pred = \
    model_training(X, y, group_kfold, s, lgb_params, 
    'binary_logloss', plt_iter=True, X_test=X_test)

In [ ]:
plt.figure(figsize=(10, 10));
sns.barplot(x="importance", y="feature", data=feature_importance[:30])
plt.title('Feature Importnace')

In [ ]:
import warnings
warnings.filterwarnings("ignore")
import numpy as np
import pandas as pd
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingRegressor, HistGradientBoostingClassifier, RandomForestClassifier
from sklearn.model_selection import KFold, GroupKFold
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import log_loss
from tqdm.notebook import tqdm
import glob
import os
import gc
import xgboost as xgb

train = tourney
test = test

xgb_params= {
        "objective": "binary:logistic",
        "max_depth": 2,
        "learning_rate": 0.1,
        "colsample_bytree": 0.8,
        "subsample": 0.8,
        #"reg_alpha" : 0,
        "min_child_weight": 30,
        "n_jobs": -1,
        "seed": 2021,
        'tree_method': "gpu_hist",
        "gpu_id": 0 ,
        'predictor': 'gpu_predictor'
    }

y = train["result"]
s = train["Season"]
X = train.drop(['Season','TeamID1','TeamID2','result'], axis=1)

X_test = test.drop(['ID', 'Season','TeamID1','TeamID2'], axis=1)

train_oof = np.zeros((X.shape[0],))
test_preds = 0
train_oof.shape

NUM_FOLDS = 5
kf = GroupKFold(n_splits=NUM_FOLDS)
max_iter = 550

for f, (train_ind, val_ind) in tqdm(enumerate(kf.split(X, y, s))):
        #print(f'Fold {f}')
        train_df, val_df = X.iloc[train_ind], X.iloc[val_ind]
        train_target, val_target = y.iloc[train_ind], y.iloc[val_ind]
        train_df_xgb = xgb.DMatrix(train_df, label=train_target)
        val_df_xgb = xgb.DMatrix(val_df, label=val_target)
        
        model = HistGradientBoostingClassifier(max_iter=max_iter, validation_fraction=None, learning_rate=0.01, max_depth=2, min_samples_leaf=32)
        model1 = RandomForestClassifier()
        model2 = LogisticRegression(C=1,solver='liblinear')
        model4 = xgb.train(xgb_params, train_df_xgb, 1000)

        model =  model.fit(train_df, train_target)
        model1 =  model1.fit(train_df, train_target)
        model2 =  model2.fit(train_df, train_target)

        
        temp_oof = (model.predict_proba(val_df)[:,1] + \
                    model1.predict_proba(val_df)[:,1] + \
                    model2.predict_proba(val_df)[:,1] + \
#                     model3.predict_proba(val_df)[:,1] + \
                    model4.predict(val_df_xgb)) / 4
        temp_test = (model.predict_proba(X_test)[:,1] \
                     + model1.predict_proba(X_test)[:,1] \
                     + model2.predict_proba(X_test)[:,1] \
#                      + model3.predict_proba(X_test)[:,1] \
                     + model4.predict(xgb.DMatrix(X_test))) / 4

        train_oof[val_ind] = temp_oof
        test_preds += temp_test/NUM_FOLDS
        
        print(log_loss(val_target, temp_oof))
        
print('CV', log_loss(y, train_oof))        
np.save('train_oof', train_oof)
np.save('test_preds', test_preds)

test = test
MSampleSubmission = pd.read_csv('../input/mens-march-mania-2022/MDataFiles_Stage2/MSampleSubmissionStage2.csv')

idx = test_preds.shape[0] //2
test_preds[idx:] = 1 - test_preds[idx:]

pred = pd.concat([test.ID, pd.Series(test_preds)], axis=1).groupby('ID')[0]\
        .mean().reset_index().rename(columns={0:'Pred'})
sub3 = MSampleSubmission.drop(['Pred'],axis=1).merge(pred, on='ID')
pred_3 = sub3['Pred']

In [ ]:
idx = test_pred.shape[0] //2
test_pred[idx:] = 1 - test_pred[idx:]

pred = pd.concat([test.ID, pd.Series(test_pred)], axis=1).groupby('ID')[0]\
        .mean().reset_index().rename(columns={0:'Pred'})
sub = MSampleSubmission.drop(['Pred'],axis=1).merge(pred, on='ID')
sub['Pred'] = sub['Pred'] * 0.25 + sub3['Pred'] * 0.75
sub.to_csv('submission.csv', index=False)
sub.head()

In [ ]:
if STAGE_1:
    rslt = pd.DataFrame()
    MTCResults_s = MTCResults.loc[MTCResults.Season >= 2015,:]
    rslt['season'] = MTCResults_s.Season
    rslt['team1'] = MTCResults_s.apply(lambda x: x.WTeamID \
                                      if x.WTeamID < x.LTeamID else x.LTeamID
                                      , axis=1)
    rslt['team2'] = MTCResults_s.apply(lambda x: x.WTeamID \
                                      if x.WTeamID > x.LTeamID else x.LTeamID
                                      , axis=1)
    rslt['wl'] = MTCResults_s.apply(lambda x: 1 if x.WTeamID < x.LTeamID else 0
                                   , axis=1)
    rslt['ID'] = rslt.apply(lambda x: str(x.season) + '_' + str(x.team1) \
                            + '_' + str(x.team2), axis=1)
    sub2 = sub.merge(rslt.loc[:,['ID','wl']], how='inner', on='ID')

    preds = []
    for i in sub2.Pred:
        preds.append([1-i, i])

    print('Test logloss is {:.5f}'.format(log_loss(sub2.wl.values, preds)))

In [ ]:
# lgb.LGBMClassifier(**params)

🚧 NOTEBOOK UNDER CONSTRUCTION 🚧


# 5 Conclusion


As I am still a beginner please feel free to suggest any improvements in the comments ! 

**References**

I got inspired from the following notebooks for the feature engineering as I know nothing in basketball.

> Very interesting notebook from @verracodeguacas,  https://www.kaggle.com/verracodeguacas/xgboost-kenpom-gagarin-ranks-modified-raddar

> Another good notebook from the 2021 competition https://www.kaggle.com/prashantkikani/ncaam-2021-diverse-model-ensemble/notebook?scriptVersionId=57044525

